#  The dataset is located at openneuro.org and its OpenNeuro Accession Number: ds003685 Files: 5398 Size: 30.52GB

## Installations:
If handling EEG / brain -imaging data for the first time. Create a separate conda environment on your machine and install some specialized modules

### Install mne_bids and mne in one shot:
    conda create --name mne --channel conda-forge mne-bids

### Alternatively,install mne-python first, 
conda create --name=mne --channel=conda-forge mne

details here: https://mne.tools/stable/index.html

mne-python is the most popular open source tool for EEG. It is quite mature, allows for good interactive visualization of brain imaging data including EEG
If using an M1 mac, be mindful of arm64 specific environment; if planning to use built-in GPU for keras / tensorflow DL

### install openneuro.py:
pip install openneuro-py

## Exploring ds03685 dataset with mne_bids

In [14]:
import os
import os.path as op
import openneuro
import mne
import matplotlib as pltA
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report
import pandas as pd
from tqdm import tqdm

In [2]:
# .. note: If the keyword argument include is left out of
#          ``openneuro.download``, the whole dataset will be downloaded.
#          We're just using data from one subject to reduce the time
#          it takes to run the example.

dataset = 'ds003685'
subject = '01'
bids_root = op.join(op.dirname(sample.data_path()), dataset)
if not op.isdir(bids_root):
    os.makedirs(bids_root)

In [6]:
# Download one subject's data from each dataset
openneuro.download(dataset=dataset, target_dir=bids_root,
                   include=[f'sub-{subject}'])


👋 Hello! This is openneuro-py 2022.1.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds003685 …
👉 Retrieving up to 94 files (5 concurrent downloads).
✅ Finished downloading ds003685.

🧠 Please enjoy your brains.



README:   0%|          | 0.00/891 [00:00<?, ?B/s]

dataset_description.json: 0.00B [00:00, ?B/s]

participants.tsv: 0.00B [00:00, ?B/s]

CHANGES: 0.00B [00:00, ?B/s]

...

sub-01_ses-session2_task-memory_run-4_eeg.json: 0.00B [00:00, ?B/s]

sub-01_ses-session3_task-eyesopen_run-2_eeg.vhdr: 0.00B [00:00, ?B/s]

sub-01_ses-session3_task-music_run-5_events.tsv: 0.00B [00:00, ?B/s]

sub-01_ses-session2_task-mathematic_run-3_eeg.vhdr: 0.00B [00:00, ?B/s]

sub-01_ses-session3_task-music_run-5_eeg.eeg: 0.00B [00:00, ?B/s]

In [15]:
print_dir_tree(bids_root, max_depth=4)

|ds003685/
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.tsv
|--- eeg/
|--- sub-01/
|------ ses-session1/
|--------- eeg/
|------------ sub-01_ses-session1_task-eyesclosed_run-1_channels.tsv
|------------ sub-01_ses-session1_task-eyesclosed_run-1_eeg.eeg
|------------ sub-01_ses-session1_task-eyesclosed_run-1_eeg.json
|------------ sub-01_ses-session1_task-eyesclosed_run-1_eeg.vhdr
|------------ sub-01_ses-session1_task-eyesclosed_run-1_eeg.vmrk
|------------ sub-01_ses-session1_task-eyesclosed_run-1_events.tsv
|------------ sub-01_ses-session1_task-eyesopen_run-2_channels.tsv
|------------ sub-01_ses-session1_task-eyesopen_run-2_eeg.eeg
|------------ sub-01_ses-session1_task-eyesopen_run-2_eeg.json
|------------ sub-01_ses-session1_task-eyesopen_run-2_eeg.vhdr
|------------ sub-01_ses-session1_task-eyesopen_run-2_eeg.vmrk
|------------ sub-01_ses-session1_task-eyesopen_run-2_events.tsv
|------------ sub-01_ses-session1_task-mathematic_run-3_channels.tsv
|---

In [16]:
print(make_report(bids_root))

Summarizing participants.tsv /Users/asgnxt/mne-miniconda/mne_data/ds003685/participants.tsv...
The participant template found: comprised of 28 male and 32 female participants;
handedness were all unknown;
ages ranged from 18.0 to 28.0 (mean = 20.0, std = 1.89; 5 with unknown age)
 The A test-retest resting and cognitive state EEG dataset dataset was created
by Yulin Wang, Wei Duan, Lihong Ding, Debo Dong, and Xu Lei and conforms to BIDS
version 1.2. This report was generated with MNE-BIDS
(https://doi.org/10.21105/joss.01896). The dataset consists of 1 participants
(comprised of 28 male and 32 female participants; handedness were all unknown;
ages ranged from 18.0 to 28.0 (mean = 20.0, std = 1.89; 5 with unknown age)) and
3 recording sessions: session1, session2, and session3.


/var/folders/f6/xkyty3nj32785_c9xjmhv0p80000gn/T/ipykernel_18343/3390368180.py:1: RuntimeWarning: No *scans.tsv files found. Currently, we do not generate a report without the scans.tsv files.
  print(make_report(bids_root))


In [17]:
datatype = 'eeg'
session = 'session1' # change to session2, session3 after done with first one
task = 'music' # change when finished
suffix = 'eeg'
bids_path = BIDSPath(task=task,
                     suffix=suffix, datatype=datatype, root=bids_root)

In [6]:
print(bids_path.match())

[BIDSPath(
root: /Users/asgnxt/mne-miniconda/mne_data/ds003685
datatype: eeg
basename: sub-01_ses-session1_task-music_run-5_eeg.eeg), BIDSPath(
root: /Users/asgnxt/mne-miniconda/mne_data/ds003685
datatype: eeg
basename: sub-01_ses-session1_task-music_run-5_eeg.vhdr), BIDSPath(
root: /Users/asgnxt/mne-miniconda/mne_data/ds003685
datatype: eeg
basename: sub-01_ses-session1_task-music_run-5_eeg.vmrk), BIDSPath(
root: /Users/asgnxt/mne-miniconda/mne_data/ds003685
datatype: eeg
basename: sub-01_ses-session2_task-music_run-5_eeg.eeg), BIDSPath(
root: /Users/asgnxt/mne-miniconda/mne_data/ds003685
datatype: eeg
basename: sub-01_ses-session2_task-music_run-5_eeg.vhdr), BIDSPath(
root: /Users/asgnxt/mne-miniconda/mne_data/ds003685
datatype: eeg
basename: sub-01_ses-session2_task-music_run-5_eeg.vmrk), BIDSPath(
root: /Users/asgnxt/mne-miniconda/mne_data/ds003685
datatype: eeg
basename: sub-01_ses-session3_task-music_run-5_eeg.eeg), BIDSPath(
root: /Users/asgnxt/mne-miniconda/mne_data/ds003685
da

In [7]:
print(bids_path)

/Users/asgnxt/mne-miniconda/mne_data/ds003685/eeg/task-music_eeg


### Read in the raw data

In [18]:
# get names of the .vhdr files
basenames = []
for i in tqdm(range(0, 8)):
    if bids_path.match()[i].basename[-4:] == 'vhdr':
        basenames.append(bids_path.match()[i])

100%|████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 217.75it/s]


In [19]:
# iterate through all files and create preprocessed dataframes
df = pd.DataFrame()
for basename in tqdm(basenames):
    raw = read_raw_bids(bids_path=basename, verbose=False)
    raw.load_data()
    raw = raw.filter(1,40)
    raw = raw.resample(100, npad="auto") 
    raw_data = raw.get_data()
    df1 = pd.DataFrame(raw_data)
    df = pd.concat([df, df1], sort=True)

  0%|                                                                                 | 0/3 [00:00<?, ?it/s]

Reading 0 ... 149999  =      0.000 ...   299.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 sec)



 33%|████████████████████████▎                                                | 1/3 [00:00<00:00,  2.45it/s]

Reading 0 ... 149999  =      0.000 ...   299.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 sec)



 67%|████████████████████████████████████████████████▋                        | 2/3 [00:00<00:00,  2.46it/s]

Reading 0 ... 149999  =      0.000 ...   299.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 sec)



100%|█████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.38it/s]


In [20]:
basenames = []
bids_path.match()[8].basename[-4:] == 'vhdr'
basenames.append(bids_path.match()[8])

In [69]:
print(raw.info['subject_info'])

{'his_id': 'sub-01', 'age': 'n/a', 'sex': 1, 'Height': '178', 'Weight': '58', 'SAS_1stVisit': '30', 'SDS_1stVisit': '38', 'ESS_1stVisit': '5', 'SAS_2ndVisit': '33', 'SDS_2ndVisit': '35', 'ESS_2ndVisit': '3', '(1)Discontinuity of Mind_session1': '5', '(2)Theory of Mind_session1': '8', '(3)Self_session1': '9', '(4)Planning_session1': '9', '(5)Sleepiness_session1': '1', '(6)Comfort_session1': '6', '(7)Somatic Awareness_session1': '5', '(8)Health Concern_session1': '10', '(9)Visual Thought_session1': '9', '(10)Verbal Thought_session1': '4', 'KSS_session1': '5', 'PANAS-P_session1': '32', 'PANAS-N_session1': '12', 'Subtraction_Correct/Not correct_session1': 'n/a', 'Subtraction_The final number_session1': 'n/a', 'Music Times of singing music lyrics_session1': 'n/a', 'Memory_Times of recall_session1': 'n/a', '(1) Discontinuity of Mind_session2': '1', '(2) Theory of Mind_session2': '9', '(3)Self_session2': '6', '(4)Planning_session2': '10', '(5)Sleepiness_session2': '1', '(6)Comfort_session2': 

In [70]:
print(raw.info['line_freq'])

50.0


In [71]:
print(raw.info['sfreq'])

100.0


In [13]:
raw.plot(block=True)

Opening raw-browser...
Using pyopengl with version 3.1.6
Closing raw-browser...
Channels marked as bad:
none
